In [1]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf

In [8]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf

# 定义贝叶斯神经网络
class BayesianNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BayesianNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 初始化贝叶斯神经网络
input_dim = 1
hidden_dim = 50
output_dim = 1
bnn = BayesianNN(input_dim, hidden_dim, output_dim)

# 定义模型和导数
def model(x, y):
    priors = {
        'fc1.weight': dist.Normal(0, 1).expand([hidden_dim, input_dim]).to_event(2),
        'fc1.bias': dist.Normal(0, 1).expand([hidden_dim]).to_event(1),
        'fc2.weight': dist.Normal(0, 1).expand([hidden_dim, hidden_dim]).to_event(2),
        'fc2.bias': dist.Normal(0, 1).expand([hidden_dim]).to_event(1),
        'fc3.weight': dist.Normal(0, 1).expand([output_dim, hidden_dim]).to_event(2),
        'fc3.bias': dist.Normal(0, 1).expand([output_dim]).to_event(1),
    }
    lifted_module = pyro.random_module("module", bnn, priors)
    lifted_reg_model = lifted_module()
    with pyro.plate("data", len(x)):
        prediction = lifted_reg_model(x).squeeze(-1)
        pyro.sample("obs", dist.Normal(prediction, 0.1).to_event(1), obs=y)

def guide(x, y):
    priors = {
        'fc1.weight': dist.Normal(0, 1).expand([hidden_dim, input_dim]).to_event(2),
        'fc1.bias': dist.Normal(0, 1).expand([hidden_dim]).to_event(1),
        'fc2.weight': dist.Normal(0, 1).expand([hidden_dim, hidden_dim]).to_event(2),
        'fc2.bias': dist.Normal(0, 1).expand([hidden_dim]).to_event(1),
        'fc3.weight': dist.Normal(0, 1).expand([output_dim, hidden_dim]).to_event(2),
        'fc3.bias': dist.Normal(0, 1).expand([output_dim]).to_event(1),
    }
    lifted_module = pyro.random_module("module", bnn, priors)
    return lifted_module()

# 生成一些示例数据
train_x = torch.linspace(0, 1, 10).unsqueeze(-1)
train_y = torch.sin(train_x * (2 * torch.pi)) + 0.2 * torch.randn(train_x.size())

# 设置Pyro优化器和推断算法
optim = Adam({"lr": 0.01})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

# 训练贝叶斯神经网络
num_iterations = 1000
for _ in range(num_iterations):
    loss = svi.step(train_x, train_y)

# 使用贝叶斯神经网络进行贝叶斯优化
def bnn_predict(x):
    sampled_models = [guide(None, None)() for _ in range(100)]
    yhats = torch.stack([model(x).forward(x).detach() for model in sampled_models])
    return yhats.mean(0), yhats.std(0)

# 定义获取函数（使用均值和标准差）
def acquisition_function(x):
    mean, std = bnn_predict(x)
    return mean + 0.1 * std

# 优化获取函数
bounds = torch.stack([torch.zeros(1), torch.ones(1)])
candidate, acq_value = optimize_acqf(
    UpperConfidenceBound(bnn_predict(train_x), beta=0.1), bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

print(f"Optimal input: {candidate}, acquisition value: {acq_value}")



C:\Users\yuanq\Anaconda3\Lib\site-packages\pyro\primitives.py:491: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  warnings.warn(


TypeError: BayesianNN.forward() missing 1 required positional argument: 'x'

In [6]:
# 初始化贝叶斯神经网络
input_dim = 1
hidden_dim = 50
output_dim = 1
bnn = BayesianNN(input_dim, hidden_dim, output_dim)

# 生成一些示例数据
train_x = torch.linspace(0, 1, 10).unsqueeze(-1)
train_y = torch.sin(train_x * (2 * torch.pi)) + 0.2 * torch.randn(train_x.size())

# 设置Pyro优化器和推断算法
optim = Adam({"lr": 0.01})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

# 训练贝叶斯神经网络
num_iterations = 1000
for _ in range(num_iterations):
    loss = svi.step(train_x, train_y)


C:\Users\yuanq\Anaconda3\Lib\site-packages\pyro\primitives.py:491: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  warnings.warn(


In [4]:
# 使用贝叶斯神经网络进行贝叶斯优化
def bnn_predict(x):
    sampled_models = [guide(None, None)() for _ in range(100)]
    yhats = torch.stack([model(x).detach() for model in sampled_models])
    return yhats.mean(0), yhats.std(0)

# 定义获取函数（使用均值和标准差）
def acquisition_function(x):
    mean, std = bnn_predict(x)
    return mean + 0.1 * std

# 优化获取函数
bounds = torch.stack([torch.zeros(1), torch.ones(1)])
candidate, acq_value = optimize_acqf(
    acquisition_function, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

print(f"Optimal input: {candidate}, acquisition value: {acq_value}")

TypeError: BayesianNN.forward() missing 1 required positional argument: 'x'